In [1]:
from collections import defaultdict, OrderedDict
from datetime import date, datetime, time, timedelta
from gridfs import GridFS
from pymongo import MongoClient

from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from ajna_commons.utils.images import get_cursor, get_imagens_recortadas

db = MongoClient(host=MONGODB_URI)[DATABASE]
fs = GridFS(db)

2019-03-28 19:46:43,925 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  /home/ivan/pybr/ajna/padma/ajna_commons/flask/error.log
Fazendo log de atividade no arquivo  /home/ivan/pybr/ajna/padma/ajna_commons/flask/access.log


In [2]:
FALTANTES = {'metadata.contentType': 'image/jpeg',
             'metadata.carga.atracacao.escala': None}

DELTA_VAZIO = 3
DELTA_IMPORTACAO = -5
DELTA_EXPORTACAO = +10

def carga_faltantes(data_inicio, data_fim, campo):
    dict_faltantes = OrderedDict()
    filtro = FALTANTES
    data_fim = datetime.combine(data_fim, time.max)  # Pega atá a última hora do dia
    filtro['metadata.dataescaneamento'] = {'$gte': data_inicio, '$lte': data_fim}
    projection = ['metadata.numeroinformado', 'metadata.dataescaneamento']
    print(filtro)
    fs_cursor = db['fs.files'].find(filtro, projection=projection).sort('metadata.numeroinformado')
    for linha in fs_cursor:
        numero = linha['metadata']['numeroinformado'].lower()
        dict_faltantes[numero] = linha['_id']
    return dict_faltantes


In [3]:
def get_cursor_vazios_mongo(start, end):
    AGREGATE_VAZIO = [
    {'$match': {'dataatracacaoiso':
                {'$gte': start  - timedelta(days=DELTA_VAZIO),
                 '$lte': end + timedelta(days=DELTA_VAZIO)}}},
    {'$lookup': {
        'from': 'CARGA.EscalaManifesto',
        'localField': 'escala',
        'foreignField': 'escala',
        'as': 'lista_manifestos'
    }}, 
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'lista_manifestos.manifesto': 1}},
    {'$lookup': {
        'from': 'CARGA.ContainerVazio',
        'localField': 'lista_manifestos.manifesto',
        'foreignField': 'manifesto',
        'as': 'containers_vazios'
    }},
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'containers_vazios.container': 1,
                  'containers_vazios.manifesto': 1}},
    {'$sort': {'containers_vazios.container': 1}}
    ]

    return db['CARGA.AtracDesatracEscala'].aggregate(AGREGATE_VAZIO)

def manifestos_periodo(start, end, cursor_function):
    dict_manifestos = dict()
    cursor = cursor_function(start, end)
    for lista in cursor:
        for linha in lista['containers_vazios']:
            manifesto = linha['manifesto']
            dict_manifestos[linha['container']] = manifesto
    return dict_manifestos



In [4]:
def manifestos_unicos_containers(dict_faltantes, dict_manifestos):
    manifestos = defaultdict(list)
    for numero, _id in dict_faltantes.items():
        manifesto = dict_manifestos.get(numero)
        if manifesto is not None:
            manifestos[manifesto].append(numero)
    return manifestos




In [5]:
def mongo_find_in(db, collection: str, fields: list, in_sets: list,
                  key_field: str):
    """Realiza um find $in in_set no db.collection e retorna dict. 

    Args:
        db: conexão ao MongoDB com banco de dados selecionado "setted"

        collection: nome da coleção mongo para aplicar a "query"

        field: campo para aplicar a "filter by"

        in_set: lista ou conjunto de valores a passar para o operador "$in"

        key_field: campo para obter valores únicos, agrupa por este campo 
        colocando-o como chave do dicionário de resposta.

    Returns:
        Dicionário de resultados formatado key:value(Somente campos não nulos)
        Conjuntos de set_field

    """
    result = OrderedDict()
    filtro = {}
    for field, in_set in zip(fields, in_sets):
        filtro[field] = {'$in': list(in_set)}
    # print(filtro)
    cursor = db[collection].find(filtro)
    for linha in cursor:
        result[linha[key_field]] = {str(key): value for key, value in linha.items()
             if value is not None and key != '_id'}
    return result


In [16]:

def monta_mongo_dict(dict_manifestos_containeres, dict_faltantes):
    manifestos_set = set(dict_manifestos_containeres.keys())
    containers_set = set()
    for lista in dict_manifestos_containeres.values():
        for item in lista:
            containers_set.add(item)

    containers = mongo_find_in(db, 'CARGA.ContainerVazio', ['manifesto', 'container'],
                               [manifestos_set, containers_set], 'container')
    manifestos = mongo_find_in(db, 'CARGA.Manifesto', ['manifesto'], [manifestos_set], 'manifesto')
    manifestos_escala = mongo_find_in(db, 'CARGA.EscalaManifesto', ['manifesto'], [manifestos_set], 'manifesto')
    escalas_set = set([value['escala'] for value in manifestos_escala.values()])
    escalas = mongo_find_in(db, 'CARGA.Escala', ['escala'], [escalas_set], 'escala')
    atracacoes = mongo_find_in(db, 'CARGA.AtracDesatracEscala', ['escala'], [escalas_set], 'escala')
    mongo_dict = {}
    for container, values in containers.items():
        ldict = {'vazio': True}
        ldict['container'] = values
        manifesto = values['manifesto']
        escala = manifestos_escala[manifesto]['escala']
        ldict['manifesto'] = manifestos[manifesto]
        ldict['escala']  = escalas[escala]
        ldict['atracacao']  = atracacoes[escala]
        mongo_dict[container] = ldict
    
    return mongo_dict

In [17]:
campo = 'manifesto'
start = datetime(2017, 7, 5)
end = datetime(2017, 7, 15)
dict_faltantes = carga_faltantes(start, end, campo)
total = len(dict_faltantes.keys())
print('Total de contâineres sem %s de %s a %s: %s' %  (campo, start, end, total))

dict_manifestos = manifestos_periodo(start, end, get_cursor_vazios_mongo)
dict_manifestos_containeres = manifestos_unicos_containers(dict_faltantes, dict_manifestos)
total = len(dict_manifestos_containeres.keys())
print('Total de manifestos de %s a %s: %s' %  (start, end, total))

mongo_dict = monta_mongo_dict(dict_manifestos_containeres)

{'metadata.contentType': 'image/jpeg', 'metadata.carga.atracacao.escala': None, 'metadata.dataescaneamento': {'$gte': datetime.datetime(2017, 7, 5, 0, 0), '$lte': datetime.datetime(2017, 7, 15, 23, 59, 59, 999999)}}
Total de contâineres sem manifesto de 2017-07-05 00:00:00 a 2017-07-15 00:00:00: 446
Total de manifestos de 2017-07-05 00:00:00 a 2017-07-15 00:00:00: 6


In [19]:
dict_faltantes

OrderedDict([('almu4914556', ObjectId('5c530b781004b377f3dc2941')),
             ('almu4914601', ObjectId('5c530bf01004b377f3dc4fff')),
             ('almu4914812', ObjectId('5c530bee1004b377f3dc4f43')),
             ('almu4914936', ObjectId('5c530bc01004b377f3dc4427')),
             ('amfu3166555', ObjectId('5c530b711004b377f3dc2601')),
             ('amfu8682512', ObjectId('5c530b8d1004b377f3dc2e39')),
             ('amfu8765746', ObjectId('5c530acd1004b377f3dbf05d')),
             ('beau2213110', ObjectId('5c530ac91004b377f3dbec91')),
             ('bhcu3176527', ObjectId('5c530adf1004b377f3dbf565')),
             ('bhcu3204064', ObjectId('5c530b911004b377f3dc303d')),
             ('bmou2679382', ObjectId('5c530ad01004b377f3dbf2e9')),
             ('bmou4529550', ObjectId('5c530be81004b377f3dc4cff')),
             ('bmou5618740', ObjectId('5c530b8a1004b377f3dc2d1d')),
             ('bmou5830105', ObjectId('5c530b911004b377f3dc3071')),
             ('bsiu9627529', ObjectId('5c530bda1

In [18]:
mongo_dict

{'caiu9703069': {'atracacao': {'codigoterminal': 'brssz009',
   'dataatracacao': '10/07/2017',
   'dataatracacaoiso': datetime.datetime(2017, 7, 10, 23, 19),
   'datadesatracacao': '11/07/2017',
   'escala': '17000217977',
   'horaatracacao': '23:19:00',
   'horadesatracacao': '23:45:00',
   'localatracacao': 'embraport',
   'nometerminal': 'embraport empresa brasileira de terminais portuarios',
   'situacao': 'atracada'},
  'container': {'capacidade(pes)': '40',
   'container': 'caiu9703069',
   'manifesto': '1517701532957',
   'nometipo': "high cube (passive vents)(40'/9'6)",
   'tara(kg)': '003800,000'},
  'escala': {'cnpjagencianavegacao': '02427026000146',
   'codigoimo': '9717228',
   'dataefetivaatracacao': '10/07/2017',
   'dataprevistaatracacao': '11/07/2017',
   'dataprevistadesatracacao': '11/07/2017',
   'dataultimadesatracacao': '11/07/2017',
   'escala': '17000217977',
   'nomeagencianavegacao': 'alianca navegacao e logistica ltda.',
   'nomeembarcacao': 'cap san vincent'

In [204]:
mongo_insert_dict = {}

for container, values in containers.items():
    new_dict = values.copy()
    manifesto = new_dict['manifesto']
    escala = manifestos_escala[manifesto]['escala']
    manifesto_dict = manifestos[manifesto].copy()
    manifesto_dict['escala']  = escalas[escala]
    manifesto_dict['atracacao']  = atracacoes[escala]
    new_dict['manifesto'] = manifesto_dict
    mongo_insert_dict[container] = new_dict


In [205]:
mongo_insert_dict['caiu9703069']

{'capacidade(pes)': '40',
 'container': 'caiu9703069',
 'manifesto': {'atracacao': {'codigoterminal': 'brssz009',
   'dataatracacao': '10/07/2017',
   'dataatracacaoiso': datetime.datetime(2017, 7, 10, 23, 19),
   'datadesatracacao': '11/07/2017',
   'escala': '17000217977',
   'horaatracacao': '23:19:00',
   'horadesatracacao': '23:45:00',
   'localatracacao': 'embraport',
   'nometerminal': 'embraport empresa brasileira de terminais portuarios',
   'situacao': 'atracada'},
  'codigoagencianavegacao': '05951386001455',
  'codigoempresanavegacao': 'fr000057',
  'codigoportocarregamento': 'brssz',
  'codigoportodescarregamento': 'cntao',
  'codigoterminalcarregamento1': 'brssz009',
  'dataemissao': '07/07/2017',
  'escala': {'cnpjagencianavegacao': '02427026000146',
   'codigoimo': '9717228',
   'dataefetivaatracacao': '10/07/2017',
   'dataprevistaatracacao': '11/07/2017',
   'dataprevistadesatracacao': '11/07/2017',
   'dataultimadesatracacao': '11/07/2017',
   'escala': '17000217977'

TypeError: monta_mongo_dict() takes 0 positional arguments but 1 was given

In [209]:
mongo_dict['caiu9703069']

{'atracacao': {'codigoterminal': 'brssz009',
  'dataatracacao': '10/07/2017',
  'dataatracacaoiso': datetime.datetime(2017, 7, 10, 23, 19),
  'datadesatracacao': '11/07/2017',
  'escala': '17000217977',
  'horaatracacao': '23:19:00',
  'horadesatracacao': '23:45:00',
  'localatracacao': 'embraport',
  'nometerminal': 'embraport empresa brasileira de terminais portuarios',
  'situacao': 'atracada'},
 'container': {'capacidade(pes)': '40',
  'container': 'caiu9703069',
  'manifesto': '1517701532957',
  'nometipo': "high cube (passive vents)(40'/9'6)",
  'tara(kg)': '003800,000'},
 'escala': {'cnpjagencianavegacao': '02427026000146',
  'codigoimo': '9717228',
  'dataefetivaatracacao': '10/07/2017',
  'dataprevistaatracacao': '11/07/2017',
  'dataprevistadesatracacao': '11/07/2017',
  'dataultimadesatracacao': '11/07/2017',
  'escala': '17000217977',
  'nomeagencianavegacao': 'alianca navegacao e logistica ltda.',
  'nomeembarcacao': 'cap san vincent'},
 'manifesto': {'codigoagencianavegac

In [13]:
AGREGATE_IMPORTACAO = [
    {'$match': {'dataatracacaoiso':
                {'$gte': start  - timedelta(days=10),
                 '$lte': end}}},
    {'$lookup': {
        'from': 'CARGA.EscalaManifesto',
        'localField': 'escala',
        'foreignField': 'escala',
        'as': 'lista_manifestos'
    }}, 
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'lista_manifestos.manifesto': 1}},
    {'$lookup': {
        'from': 'CARGA.Manifesto',
        'localField': 'lista_manifestos.manifesto',
        'foreignField': 'manifesto',
        'as': 'lista_manifestos2'
    }}, 
    {'$match': {'lista_manifestos2.tipomanifesto':
                {'$in': ['lci', 'bce']}}},
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'lista_manifestos2.manifesto': 1}},
    {'$lookup': {
        'from': 'CARGA.ManifestoConhecimento',
        'localField': 'lista_manifestos2.manifesto',
        'foreignField': 'manifesto',
        'as': 'conhecimentos'
    }},
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'conhecimentos.conhecimento': 1,
                  'conhecimentos.manifesto': 1}},
    {'$lookup': {
        'from': 'CARGA.Container',
        'localField': 'conhecimentos.conhecimento',
        'foreignField': 'conhecimento',
        'as': 'containers'
    }},
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'conhecimentos.conhecimento': 1,
                  'conhecimentos.manifesto': 1,
                  'containers.conhecimento': 1,
                  'containers.container': 1
                 }}

]


In [14]:
cursor = db['CARGA.AtracDesatracEscala'].aggregate(AGREGATE_IMPORTACAO)

In [15]:
lista = list(cursor)
len(lista)

175

In [16]:
lista[0]

{'_id': ObjectId('5c5342e31004b333d727dcea'),
 'conhecimentos': [{'conhecimento': '151707128765934',
   'manifesto': '1517701441714'},
  {'conhecimento': '151707129619575', 'manifesto': '1517701449049'},
  {'conhecimento': '151707129619656', 'manifesto': '1517701449049'},
  {'conhecimento': '151707129619737', 'manifesto': '1517701449049'},
  {'conhecimento': '151707129619818', 'manifesto': '1517701449049'},
  {'conhecimento': '151707129619907', 'manifesto': '1517701449049'},
  {'conhecimento': '151707129620077', 'manifesto': '1517701449049'},
  {'conhecimento': '151707129620310', 'manifesto': '1517701449057'},
  {'conhecimento': '151707129620409', 'manifesto': '1517701449065'},
  {'conhecimento': '151707129620581', 'manifesto': '1517701449065'},
  {'conhecimento': '151707129620662', 'manifesto': '1517701449065'},
  {'conhecimento': '151707129620743', 'manifesto': '1517701449065'},
  {'conhecimento': '151707129620824', 'manifesto': '1517701449073'},
  {'conhecimento': '151707129620905',

In [17]:
for conhecimento in lista[0]['containers']:
    print(conhecimento)

{'conhecimento': '001709022013865', 'container': 'medu2079120'}
{'conhecimento': '001709022013865', 'container': 'medu2298767'}
{'conhecimento': '001709022013865', 'container': 'medu2991030'}
{'conhecimento': '001709022013865', 'container': 'mscu6709725'}
{'conhecimento': '001709022013946', 'container': 'medu2314636'}
{'conhecimento': '001709022013946', 'container': 'sczu7944560'}
{'conhecimento': '001709022014080', 'container': 'clhu2656005'}
{'conhecimento': '001709022014080', 'container': 'fscu3242959'}
{'conhecimento': '001709022014080', 'container': 'tcku3399322'}
{'conhecimento': '001709022014160', 'container': 'caxu4716851'}
{'conhecimento': '001709022014160', 'container': 'mscu4840839'}
{'conhecimento': '001709022014160', 'container': 'mscu5010224'}
{'conhecimento': '001709022014160', 'container': 'texu7393059'}
{'conhecimento': '001709022014241', 'container': 'mscu4767023'}
{'conhecimento': '001709022014241', 'container': 'mscu4783050'}
{'conhecimento': '001709022014241', 'con

In [18]:
set_conhecimentos = set()
dict_conhecimentos = dict()
for lista2 in lista:
    for linha in lista2['containers']:
        dict_conhecimentos[linha['container']] = linha['conhecimento']
        set_conhecimentos.add(linha['container'])
        
len(set_conhecimentos)

60772

In [19]:
encontrados_importacao = set_faltantes & set_conhecimentos
encontrados_importacao

{'amfu8765746',
 'beau2213110',
 'bmou2679382',
 'caiu8203840',
 'caxu6243130',
 'clhu3052150',
 'etnu1501715',
 'etnu1501802',
 'etnu1501850',
 'etnu1501865',
 'etnu1502049',
 'hasu1513262',
 'hasu4586771',
 'hlbu1631941',
 'lcru0146847',
 'medu1149223',
 'medu3100670',
 'medu3112500',
 'mscu5950018',
 'mscu6402390',
 'sudu5965453',
 'sudu7727412',
 'tghu2692604',
 'ttnu0704625',
 'ttnu4881681',
 'ttnu5169835',
 'uacu3711340',
 'uacu8063018'}

In [20]:
AGREGATE_EXPORTACAO = [
    {'$match': {'dataatracacaoiso':
                {'$gte': start  - timedelta(days=1),
                 '$lte': end + timedelta(days=DELTA_EXPORTACAO)}}},
    {'$lookup': {
        'from': 'CARGA.EscalaManifesto',
        'localField': 'escala',
        'foreignField': 'escala',
        'as': 'lista_manifestos'
    }}, 
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'lista_manifestos.manifesto': 1}},
    {'$lookup': {
        'from': 'CARGA.Manifesto',
        'localField': 'lista_manifestos.manifesto',
        'foreignField': 'manifesto',
        'as': 'lista_manifestos2'
    }}, 
    {'$match': {'lista_manifestos2.tipomanifesto':
                {'$in': ['lce']}}},
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'lista_manifestos2.manifesto': 1}},
    {'$lookup': {
        'from': 'CARGA.ManifestoConhecimento',
        'localField': 'lista_manifestos2.manifesto',
        'foreignField': 'manifesto',
        'as': 'conhecimentos'
    }},
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'conhecimentos.conhecimento': 1,
                  'conhecimentos.manifesto': 1}},
    {'$lookup': {
        'from': 'CARGA.Container',
        'localField': 'conhecimentos.conhecimento',
        'foreignField': 'conhecimento',
        'as': 'containers'
    }},
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'conhecimentos.conhecimento': 1,
                  'conhecimentos.manifesto': 1,
                  'containers.conhecimento': 1,
                  'containers.container': 1
                 }}

]


In [21]:
cursor = db['CARGA.AtracDesatracEscala'].aggregate(AGREGATE_EXPORTACAO)

In [22]:
lista = list(cursor)
len(lista)


238

In [23]:
set_conhecimentos = set()
dict_conhecimentos = dict()
for lista2 in lista:
    for linha in lista2['containers']:
        dict_conhecimentos[linha['container']] = linha['conhecimento']
        set_conhecimentos.add(linha['container'])
        
len(set_conhecimentos)

64363

In [24]:
encontrados_exportacao = set_faltantes & set_conhecimentos
encontrados_exportacao

{'amfu8682512',
 'amfu8765746',
 'bhcu3176527',
 'bhcu3204064',
 'bmou4529550',
 'bmou5830105',
 'cadu7040225',
 'caxu9342076',
 'clhu3850843',
 'cniu1185248',
 'cpsu1805229',
 'crxu8688161',
 'dfsu2364962',
 'dhdu1375757',
 'exfu0721671',
 'fciu2311362',
 'fciu4264001',
 'fciu8627770',
 'fscu3686980',
 'fscu6890359',
 'fscu9270860',
 'fscu9604741',
 'gesu3844463',
 'hasu1018417',
 'hasu1431354',
 'hasu1459217',
 'hasu1513262',
 'hasu1545125',
 'hasu4045972',
 'hasu4053860',
 'hasu4208766',
 'hasu4252748',
 'hasu4419331',
 'hasu4473760',
 'hasu4560503',
 'hlbu1631941',
 'hlbu1698040',
 'hlxu6464171',
 'hlxu8263788',
 'hlxu8314740',
 'hlxu8348864',
 'hlxu8442171',
 'ibtu1251685',
 'ihou1840712',
 'mscu6322401',
 'rfcu8211880',
 'rfcu8229344',
 'rltu2016777',
 'segu5627986',
 'sudu1481872',
 'sudu1920430',
 'sudu4869630',
 'sudu4935120',
 'sudu5163313',
 'sudu5796097',
 'sudu5848966',
 'sudu5893477',
 'sudu5965453',
 'sudu6053270',
 'sudu6555854',
 'sudu6593746',
 'sudu6764978',
 'sudu68

In [25]:
encontrados_exportacao & encontrados_vazio

{'triu0300295'}

In [27]:
encontrados_exportacao & encontrados_importacao

{'amfu8765746',
 'hasu1513262',
 'hlbu1631941',
 'sudu5965453',
 'sudu7727412',
 'ttnu0704625',
 'ttnu5169835'}

In [28]:
encontrados_vazio & encontrados_importacao

set()